In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/AYSIR2024-25

Mounted at /content/drive
/content/drive/MyDrive/AYSIR2024-25


In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("caxton_dataset_semifiltered.csv")
df.head()

,Unnamed: 0,img_path,timestamp,flow_rate,feed_rate,z_offset,target_hotend,hotend,bed,nozzle_tip_x,nozzle_tip_y,img_num,print_id,flow_rate_class,feed_rate_class,z_offset_class,hotend_class,img_mean,img_std
0,2710,caxton_dataset/print1/image-6.jpg,2020-10-02T09:43:56-03,100,100,0.0,205.0,203.47,64.90,434,514,5,1,1,1,1,1,36.130960,46.708494
1,2711,caxton_dataset/print1/image-7.jpg,2020-10-02T09:43:56-50,100,100,0.0,205.0,203.47,64.90,434,514,6,1,1,1,1,1,42.482656,50.322867
2,2712,caxton_dataset/print1/image-8.jpg,2020-10-02T09:43:56-96,100,100,0.0,205.0,203.47,64.90,434,514,7,1,1,1,1,1,37.234668,49.867208
3,2713,caxton_dataset/print1/image-9.jpg,2020-10-02T09:43:57-43,100,100,0.0,205.0,203.44,64.75,434,514,8,1,1,1,1,1,36.938919,50.725929
4,2714,caxton_dataset/print1/image-10.jpg,2020-10-02T09:43:57-90,100,100,0.0,205.0,203.44,64.75,434,514,9,1,1,1,1,1,36.490107,49.608293


In [ ]:
#dropping
#keeping: img_path, flow_rate, feed_rate, z_offset, hotend
df = df.drop(["Unnamed: 0","timestamp","target_hotend","nozzle_tip_x","nozzle_tip_y","img_num", "print_id","flow_rate_class","feed_rate_class","z_offset_class","hotend_class","img_mean","img_std"], axis = 1)
df.head()

,img_path,flow_rate,feed_rate,z_offset,hotend,bed
0,caxton_dataset/print1/image-6.jpg,100,100,0.0,203.47,64.90
1,caxton_dataset/print1/image-7.jpg,100,100,0.0,203.47,64.90
2,caxton_dataset/print1/image-8.jpg,100,100,0.0,203.47,64.90
3,caxton_dataset/print1/image-9.jpg,100,100,0.0,203.44,64.75
4,caxton_dataset/print1/image-10.jpg,100,100,0.0,203.44,64.75


In [ ]:
# only keeping print 1, print 2, print 3, and print 4 (makes up majority of data)
keep = [
    "caxton_dataset/print1/",
    "caxton_dataset/print2/",
    "caxton_dataset/print3/",
    "caxton_dataset/print4/",
]

df = df[df["img_path"].str.startswith(tuple(keep))]
df.head()

,img_path,flow_rate,feed_rate,z_offset,hotend,bed
0,caxton_dataset/print1/image-6.jpg,100,100,0.0,203.47,64.90
1,caxton_dataset/print1/image-7.jpg,100,100,0.0,203.47,64.90
2,caxton_dataset/print1/image-8.jpg,100,100,0.0,203.47,64.90
3,caxton_dataset/print1/image-9.jpg,100,100,0.0,203.44,64.75
4,caxton_dataset/print1/image-10.jpg,100,100,0.0,203.44,64.75


In [ ]:
#adding failure column
full_df = pd.read_csv("caxton_dataset_final.csv")
df["failure"] = df["img_path"].isin(full_df["img_path"]).astype(int)
df.head()

,img_path,flow_rate,feed_rate,z_offset,hotend,bed,failure
0,caxton_dataset/print1/image-6.jpg,100,100,0.0,203.47,64.90,1
1,caxton_dataset/print1/image-7.jpg,100,100,0.0,203.47,64.90,1
2,caxton_dataset/print1/image-8.jpg,100,100,0.0,203.47,64.90,1
3,caxton_dataset/print1/image-9.jpg,100,100,0.0,203.44,64.75,1
4,caxton_dataset/print1/image-10.jpg,100,100,0.0,203.44,64.75,1


In [ ]:
print("number of failures:", (df["failure"]==1).sum())
print("number of non-failures", (df["failure"]==0).sum())

number of failures: 11199
number of non-failures 125


In [ ]:
df.to_csv("final_failure_detection_dataset.csv")

difference between 1.07 millions and 946, 283 is dark imgs and parameter outliers

use no outliers img info

subtraction of filtered and final have failure 1 ~100k

final has failure 0

NEW METHOD (02/25)

In [ ]:
full_df = pd.read_csv("caxton_dataset_full.csv")
final_df = pd.read_csv("caxton_dataset_final.csv")

In [ ]:
full_df.columns

Index(['img_path', 'timestamp', 'flow_rate', 'feed_rate', 'z_offset',
       'target_hotend', 'hotend', 'bed', 'nozzle_tip_x', 'nozzle_tip_y',
       'print_id'],
      dtype='object')

In [ ]:
final_df.columns

Index(['img_path', 'timestamp', 'flow_rate', 'feed_rate', 'z_offset',
       'target_hotend', 'hotend', 'bed', 'nozzle_tip_x', 'nozzle_tip_y',
       'img_num', 'print_id', 'flow_rate_class', 'feed_rate_class',
       'z_offset_class', 'hotend_class', 'img_mean', 'img_std'],
      dtype='object')

In [ ]:
full_df = full_df.drop(["timestamp","target_hotend","nozzle_tip_x","nozzle_tip_y", "print_id"], axis = 1)
final_df = final_df.drop(["timestamp","target_hotend","nozzle_tip_x","nozzle_tip_y","img_num", "print_id","flow_rate_class","feed_rate_class","z_offset_class","hotend_class","img_mean","img_std"], axis = 1)

In [ ]:
full_df.head()

,img_path,flow_rate,feed_rate,z_offset,hotend,bed
0,caxton_dataset/print0/image-1.jpg,100,100,0.0,204.34,65.66
1,caxton_dataset/print0/image-2.jpg,100,100,0.0,204.34,65.66
2,caxton_dataset/print0/image-3.jpg,100,100,0.0,204.13,65.74
3,caxton_dataset/print0/image-4.jpg,100,100,0.0,204.13,65.74
4,caxton_dataset/print0/image-5.jpg,100,100,0.0,204.13,65.74


In [ ]:
final_df.head()

,img_path,flow_rate,feed_rate,z_offset,hotend,bed
0,caxton_dataset/print0/image-6.jpg,100,100,0.0,204.13,65.74
1,caxton_dataset/print0/image-7.jpg,100,100,0.0,204.13,65.74
2,caxton_dataset/print0/image-8.jpg,100,100,0.0,204.24,65.84
3,caxton_dataset/print0/image-9.jpg,100,100,0.0,204.24,65.84
4,caxton_dataset/print0/image-10.jpg,100,100,0.0,204.24,65.84


In [ ]:
full_df["failure"] = (1 - full_df["img_path"].isin(final_df["img_path"]).astype(int))
full_df.head()

,img_path,flow_rate,feed_rate,z_offset,hotend,bed,failure
0,caxton_dataset/print0/image-1.jpg,100,100,0.0,204.34,65.66,1
1,caxton_dataset/print0/image-2.jpg,100,100,0.0,204.34,65.66,1
2,caxton_dataset/print0/image-3.jpg,100,100,0.0,204.13,65.74,1
3,caxton_dataset/print0/image-4.jpg,100,100,0.0,204.13,65.74,1
4,caxton_dataset/print0/image-5.jpg,100,100,0.0,204.13,65.74,1


In [ ]:
len(full_df)

1272273

In [ ]:
len(final_df)

948396

In [ ]:
print("number of failures:", (full_df["failure"]==1).sum())
print("number of non-failures", (full_df["failure"]==0).sum())

number of failures: 323877
number of non-failures 948396


In [ ]:
full_df["print_folder"] = full_df["img_path"].str.extract(r"(caxton_dataset/print\d+)/")
print_counts = full_df["print_folder"].value_counts()
print(print_counts)

print_folder
caxton_dataset/print131    30196
caxton_dataset/print132    28715
caxton_dataset/print111    27489
caxton_dataset/print24     26637
caxton_dataset/print109    25813
                           ...  
caxton_dataset/print166      150
caxton_dataset/print147      125
caxton_dataset/print76       124
caxton_dataset/print161      117
caxton_dataset/print190       18
Name: count, Length: 192, dtype: int64


In [ ]:
full_df.to_csv("final_failure_detection_dataset.csv")

In [4]:
df = pd.read_csv("final_failure_detection_dataset.csv")

In [5]:
df = df[:10001]
len(df)

10001

In [6]:
df.head()

,Unnamed: 0,img_path,flow_rate,feed_rate,z_offset,hotend,bed,failure,print_folder
0,0,caxton_dataset/print0/image-1.jpg,100,100,0.0,204.34,65.66,1,caxton_dataset/print0
1,1,caxton_dataset/print0/image-2.jpg,100,100,0.0,204.34,65.66,1,caxton_dataset/print0
2,2,caxton_dataset/print0/image-3.jpg,100,100,0.0,204.13,65.74,1,caxton_dataset/print0
3,3,caxton_dataset/print0/image-4.jpg,100,100,0.0,204.13,65.74,1,caxton_dataset/print0
4,4,caxton_dataset/print0/image-5.jpg,100,100,0.0,204.13,65.74,1,caxton_dataset/print0


In [7]:
print_counts = df["print_folder"].value_counts()
print(print_counts)

print_folder
caxton_dataset/print1    5901
caxton_dataset/print0    3085
caxton_dataset/print2    1015
Name: count, dtype: int64


In [8]:
df.drop(["print_folder"], axis = 1)

,Unnamed: 0,img_path,flow_rate,feed_rate,z_offset,hotend,bed,failure
0,0,caxton_dataset/print0/image-1.jpg,100,100,0.00,204.34,65.66,1
1,1,caxton_dataset/print0/image-2.jpg,100,100,0.00,204.34,65.66,1
2,2,caxton_dataset/print0/image-3.jpg,100,100,0.00,204.13,65.74,1
3,3,caxton_dataset/print0/image-4.jpg,100,100,0.00,204.13,65.74,1
4,4,caxton_dataset/print0/image-5.jpg,100,100,0.00,204.13,65.74,1
...,...,...,...,...,...,...,...,...
9996,9996,caxton_dataset/print2/image-1011.jpg,131,119,0.02,195.45,65.33,1
9997,9997,caxton_dataset/print2/image-1012.jpg,131,119,0.02,195.45,65.33,1
9998,9998,caxton_dataset/print2/image-1013.jpg,131,119,0.02,195.45,65.33,1
9999,9999,caxton_dataset/print2/image-1014.jpg,131,119,0.02,195.45,65.33,1


In [ ]:
df.head()

In [9]:
df.to_csv("failure_detection_dataset.csv")